In [2]:
import numpy as np
import random
import gc
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
# Definicion de hiperparametros:

batch_size = 128 #tamaño de lotes de entrenamiento

hidden_dim_1 = 20 #tamaño de la capa 1
hidden_dim_2 = 10 #tamaño de la capa 2

lr = 0.01 #tasa de aprendizaje
n_epochs = 10 #reiteraciones sobre la base de datos de entrenamiento completa

In [4]:
# Descarga del dataset:

train_dataset = datasets.MNIST("./data", train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST("./data", train=False, download=True, transform=transforms.ToTensor())

In [31]:
# Carga de los datos: 

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8)
test_data = torch.utils.data.DataLoader(test_dataset)

In [6]:
# Comprobacion del dispositivo de ejecucion: 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
# Definicion de la arquitectura modelo:

encoder_arch = nn.Sequential(
    nn.Flatten(), #tranforma el array de entrada (28,28) en un tensor (28*28,1)
    nn.Linear(28*28, hidden_dim_1),
    nn.ReLU(),
    nn.Linear(hidden_dim_1, hidden_dim_2),
    nn.Sigmoid()
)

In [20]:
# Definicion del modelo:

class MNIST_classifier(nn.Module):
    def __init__(self, encoder_arch):
        super(MNIST_classifier, self).__init__()
        self.forward_arch = encoder_arch
    def forward(self, data):
        out = self.forward_arch(data)
        return out

In [21]:
# Inicializacion del modelo:

clasificador = MNIST_classifier(encoder_arch=encoder_arch).to(device)


In [22]:
# Funcion de pérdida:
loss_function = nn.CrossEntropyLoss()

In [23]:
# Optimizador: 
optimizer = optim.Adam(clasificador.parameters(), lr = lr, betas=(0.9, 0.999))

In [24]:
# Funcion de evaluacion del modelo sobre toda la base de datos de evaluzacion

def model_eval(model, device, test_data):
    model.eval()
    correct = 0
    for image, label in test_data:
        image, label = image.to(device), label.to(device)
        prediction = model(image)
        if int(torch.argmax(prediction)) == int(label):
            correct = correct + 1
    return correct

In [25]:
# Funcion de entrenamiento: 

def model_train(model, train_data, test_data, n_epochs, optimizer, loss_function):
    model.train()
    for epoch in range(n_epochs):
        #correct = model_eval(model, device, test_data)
        #print(f"Epoch {epoch} ----> Correct guesses: {correct}/{len(test_data)}")
        model.train()
        for batch_id, (image, label) in enumerate(train_data):
            image, label = image.to(device), label.to(device)
            optimizer.zero_grad()
            model_output = model(image)
            loss = loss_function(model_output, label)
            
            loss.backward()
            optimizer.step()
            
            if batch_id % 1000 == 0:
                print(f"Epoch {epoch} [{batch_id * len(image)}/{len(train_data.dataset)}] Loss: {loss.item():6f}")

In [14]:
for images, labels in test_data:
    #print(f"Batch {batch_idx}")
    print(f"Images shape: {images.shape}")  # Ej: torch.Size([32, 1, 28, 28])
    print(f"Labels shape: {labels.shape}")  # Ej: torch.Size()

Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images shape: torch.Size([1, 1, 28, 28])
Labels shape: torch.Size([1])
Images

In [33]:
model_train(clasificador,train_data,test_data, n_epochs,optimizer,loss_function)

Epoch 0 [0/60000] Loss: 2.300960
Epoch 1 [0/60000] Loss: 1.527489
Epoch 2 [0/60000] Loss: 1.498331
Epoch 3 [0/60000] Loss: 1.498031
Epoch 4 [0/60000] Loss: 1.508165
Epoch 5 [0/60000] Loss: 1.519737
Epoch 6 [0/60000] Loss: 1.500075
Epoch 7 [0/60000] Loss: 1.524228
Epoch 8 [0/60000] Loss: 1.499425
Epoch 9 [0/60000] Loss: 1.504545


In [34]:
correct = model_eval(clasificador, device, test_data)
print(f"Correct guesses: {correct}/{len(test_data)}")

Correct guesses: 9443/10000


In [32]:
# Funcion para resetear el modelo

def reset_weights(m):
    if hasattr(m, "reset_parameters"):
        m.reset_parameters()

clasificador.apply(reset_weights)

MNIST_classifier(
  (forward_arch): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=20, bias=True)
    (2): ReLU()
    (3): Linear(in_features=20, out_features=10, bias=True)
    (4): Sigmoid()
  )
)